In [4]:
import sqlalchemy
import pandas as pd
import yahooquery as yq
import sqlite3 as lite

In [5]:
engine = sqlalchemy.create_engine('sqlite:///' + "SEC_Filings.db", execution_options={"sqlite_raw_colnames": True})
df_ticker = pd.read_sql_table('Ticker_Table', engine)
df_fin_bal = pd.DataFrame()
df_fin_inc = pd.DataFrame()
bal_vars = ['asOfDate', 'CashAndCashEquivalents','TotalAssets','TotalDebt','TangibleBookValue','TotalCapitalization']
inc_vars = ['asOfDate', 'BasicEPS']
# bal_sheet_error = []
bal_sheet_error_tickers = []
inc_stat_error_tickers = []

for ticker in df_ticker['Ticker']:
    tick = yq.Ticker(ticker)
    df_bal_tick = tick.balance_sheet(frequency='a', trailing=False)
    df_inc_tick = tick.income_statement(frequency='a', trailing=False)

    try:
        df_bal_tick = df_bal_tick[bal_vars]
        df_fin_bal = pd.concat([df_fin_bal, df_bal_tick], axis=0)

    except:
        if type(df_bal_tick) == pd.DataFrame:
            # bal_sheet_error.append(f'{ticker}: Missing columns {[var for var in bal_vars if var not in df_bal.columns]}')
            bal_sheet_error_tickers.append(ticker)
        else:
            # bal_sheet_error.append(f'{ticker} Balance sheet not Available')
            bal_sheet_error_tickers.append(ticker)

    try:
        df_inc_tick = df_inc_tick[inc_vars]
        df_fin_inc = pd.concat([df_fin_inc, df_inc_tick], axis=0)

    except:
        if type(df_inc_tick) == pd.DataFrame:
            # print([var for var in inc_vars if var not in df_inc.columns])
            inc_stat_error_tickers.append(ticker)
        else:
            inc_stat_error_tickers.append(ticker)
            
df_fin_bal.reset_index(drop=False, inplace=True)
df_fin_inc.reset_index(drop=False, inplace=True)

# Only take rows with if matched in both dataframes
df_fin_joined = df_fin_inc.merge(df_fin_bal, how='inner', on=['symbol', 'asOfDate'])

In [7]:
df_fin_cleaned = pd.DataFrame()
# backfill missing values based on company
for ticker in df_fin_joined['symbol'].unique():
    df_fin_cleaned = pd.concat([df_fin_cleaned, df_fin_joined.loc[df_fin_joined['symbol']==ticker].sort_values(
                        by='asOfDate', ascending=True).fillna(method='ffill')])

# Remove rows with incomplete data
missing_ticks = df_fin_cleaned[df_fin_cleaned.isnull().any(axis=1)]['symbol'].unique()
df_fin_cleaned = df_fin_cleaned.loc[~df_fin_cleaned['symbol'].isin(missing_ticks)].reset_index(drop=True)

df_fin_cleaned = df_ticker.merge(df_fin_cleaned, how="right", left_on="Ticker", right_on="symbol").drop("symbol", axis=1)

# Add to database
conn = lite.connect('Financial.db')
conn.close()
engine_fin = sqlalchemy.create_engine('sqlite:///' + 'Financial.db', execution_options={"sqlite_raw_colnames": True})
df_fin_cleaned.to_sql("FinancialDataJoined", engine_fin, if_exists='replace', index=False)

1608